In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai

# ✅ 1. Set your OpenAI API key securely
api_key = "your_openai_key"

# ✅ 2. Load dataset
df = pd.read_excel("cicdgitops.xlsx")
contexts = df['Context'].astype(str).tolist()

# ✅ 3. Generate dense embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(contexts)

# ✅ 4. Create FAISS index for fast retrieval
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# ✅ 5. Define retriever function
def rag_query(question, top_k=5):
    query_vec = model.encode([question])
    D, I = index.search(np.array(query_vec), top_k)
    retrieved_contexts = [contexts[i] for i in I[0]]
    context_block = "\n\n".join(retrieved_contexts)
    
    prompt = f"""
You are an expert in DevOps. Based on the following literature evidence, answer the question:

CONTEXT:
{context_block}

QUESTION:
{question}

ANSWER:"""
    return prompt

# ✅ 6. Define generator using OpenAI (v1.x syntax)
client = openai.OpenAI(api_key=api_key)

def generate_answer(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content

# ✅ 7. Interactive input from user
question = input("❓ Enter your DevOps-related question: ")

# Generate RAG prompt
prompt = rag_query(question)
print("\n📌 Prompt sent to GPT:\n")
print(prompt)

# Generate answer
print("\n✅ GPT-Generated Answer:\n")
answer = generate_answer(prompt)
print(answer)



❓ Enter your DevOps-related question:  is it possible to leverage gitops without kubernetes?



📌 Prompt sent to GPT:


You are an expert in DevOps. Based on the following literature evidence, answer the question:

CONTEXT:
GitOps built on native Kubernetes mechanisms.

GitOps deployment tool for Kubernetes

GitOps deployment tool for Kubernetes

A GitOps continuous delivery tool for Kubernetes.

GitOps delivery tool for Kubernetes

QUESTION:
is it possible to leverage gitops without kubernetes?

ANSWER:

✅ GPT-Generated Answer:

The literature evidence provided does not contain information on whether GitOps can be leveraged without Kubernetes.
